<p style="font-size:30px; text-align:center; line-height:120%">
<br>
COMS W 4111-02, H02, V02<br>
Homework 4
</p>

# Introduction

- This homework has two learning objectives:
    - Learning basic concepts for using a graph database to model core information (sample movie information database) and augment with social information (Person, Fan, Follows, Likes, ...)
    - Learn the basic concepts behind Redis by implementing a simple version of one of the most common Redis solution usages -- [result caching.](https://redislabs.com/blog/query-caching-redis/)
    
    
- Being able to discuss working on these objectives and having implemented some functions is a cool topics to discuss on job and internship interviews, and looks cool on your resume.
    
    
- The notebook contains a section on setting up software. The section is a subset of the setup for the final exam. Completing the assignment decreases the chance of your having setup problems while working on the final exam.


- The assignment seems to require significant programming when you first review it. This is not the case. Most of what you have to implement is a set of DB operations/calls to Neo4j and Redis. The helper code in section 2 provides almost ll of the code you need to execute the queries. 

# Setup

- In previous semesters, I required students to install Redis and Neo4j software, including server SW on their laptops.
    - Understanding how to install, configure and use SW is a valuable skill in fields that use computers in an advanced way: CS, data science, IEOR analysis, etc.
    - SW installation is tedious and causes annoying issues for some students. 
    - For this assignment, you do not need to install the Redis or Neo4j products. You can use free evaluation versions available via [Platform-as-a-Service (PaaS)](https://en.wikipedia.org/wiki/Platform_as_a_service).
    - The PaaS approach significantly decreases the time and complexity for installation and setup.
    - Saying you have used PaaS DBs also sounds cool on CVs and in interviews.
    
    
- Setting Up the Environment
    - You will need to use Neo4j and Redis to complete this homework assignment.
    - Neo4j
        - Please follow the instructions in the previous [recorded makeup lecture](https://cvn.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=169786d3-c838-4f2c-ada6-ab1800d0e654) for setting up a cloud version of Neo4j.
        - You must have installed the sample movie database. The recorded lecture has instructions.
        - You must install the [Neo4j REST Client](https://docs.graphenedb.com/docs/python#section-neo4j-rest-client). The easiest way to do this is:
            - Go to the ```File``` menu option in the Notebook and select ```Open.``` This will open a new file explorer.
            - On the upper right hand corner, click on ```New``` and choose terminal. This will open a terminal window.
            - In the terminal window, enter ```pip install neo4jrestclient```
    - Redis:
        - Please follow the instructions in the [recorded makeup lecture](https://cvn.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=169786d3-c838-4f2c-ada6-ab1800d0e654) for setting up a cloud version of Redis.
        - You do not need to install or configure any data.
        - The Anaconda environment comes with a Redis client.
        
        
- Section 2 tests your setup.

# Environment Test


In [1]:
# Import some useful packages. Both are an integral part of Conda/Ananconda.
import json
import pandas

## Neo4j

### Concept

- Neo4j is a graph database.


- Many databases, including the relational model, have limited support for complex, dynamic relationships between entities. The limitations are both functional and performance.


- Relationships are a first class concept in graph databases.
    - This data manipulation language directly supports graph concepts, e.g. paths.
    - Graph DBs internal implementation (indexes, memory storage, etc) focuses on supporting graphs. This enables DBs like Neo4j to traverse relationships orders-of-magnitude faster than other models.
    
    
- Very simply, a graph database is a database designed to treat the relationships between data as equally important to the data itself.

### Setup

In [2]:
# Import the client side connection package for Neo4j.
# There are many client side packages for Neo4j, which is also
# true for MySQL and most databases. We are importing one of the
# simpler client connection packages.
# https://neo4j-rest-client.readthedocs.io/en/latest/info.html
import neo4jrestclient
from neo4jrestclient.client import GraphDatabase
from neo4jrestclient import client

In [3]:
# We are using HTTPS because the API calls are going over a public network.
# HTTPS requires setting up and configuring certificates, which can be 
# difficult and confusing. Since we are not configuring certificates,
# the neo4jrestclient code issues warning messages, and these become
# distracting. The code below supresses the warnings.
#
# DO NOT EVER DO THIS IN A REAL SOLUTION.
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
neo4jrestclient.options.VERIFY_SSL = False 

# The code below is the Neo4j REST Client equivalent to the connection
# setup we have done for MySQL.
#
# YOU MUST USE THE URL, USER ID AND PASSWORD THAT YOU SET UP WHEN
# CONFIGURING YOUR PAAS VERSION OF NEO4J. WE WILL CHANGE THIS INFO
# WHEN TESTING YOUR HOMEWORK.
neo4j_host = "https://hobby-bmknbolbhobhgbkecbgceedl.dbs.graphenedb.com:24780/db/data/"
neo4j_user = "shangzi"
neo4j_password = "b.W9QG12Hg97Vh.afEaMLwQLqY745LF"

# gdb is a global variable in the notebook code. The object plays the same
# role as the connection object we have used in MySQL. Unlike the MySQL connection,
# the connection does have graph oriented operations. By and large, however, we
# simple use the query() method in this homework.
gdb = GraphDatabase(neo4j_host, username = neo4j_user, password = neo4j_password)

In [47]:
# The data that comes back from a REST query to Neo4j looks weird if you
# are not expert in graphs and REST. This is a helper method that extracts
# the application data from a raw REST response.
#
# Do not worry about the details.
#
def get_stuff(d):
    result = {}
    result['id'] = d['metadata']['id']
    result['labels'] = d['metadata'].get('labels', None)
    result['properties'] = d['data']
    return result


__Testing Getting Tom Hanks__

Match all nodes where the name on that node is tom hanks and return all of the properties of that node

In [48]:
# Submit the query.
result = gdb.query("MATCH (n:Person) where n.name='Tom Hanks' return n")

# This is similar to having to interate the the list of rows returned
# by a MySQL query through a connection. The structure of the returned
# data is more complex.

print("The following records matched the query")
for record in result:
    n = get_stuff(record[0])
    print("Node = ", json.dumps(n, indent=2))

The following records matched the query
Node =  {
  "id": 71,
  "labels": [
    "Person"
  ],
  "properties": {
    "born": 1956,
    "name": "Tom Hanks"
  }
}


- Executing the same query in the Neoj console, which is analogous to MySQL Workbench produces:

<img src="./simple_neo4j_example.png">


- The Neo4j console ran the query under the covers, received the response and displayed the information graphically.

__Testing a More Complex Thing__

In [49]:
# Run the query
result = gdb.query("MATCH (n:Person)-[r:ACTED_IN]->(m:Movie) where n.name='Tom Hanks' return n,r,m")

# Go through every "row" in the query result.
for record in result:
    
    # The structure of a row in a result in Neo4j is more complex. The return clause listed three things: 
    # start node, relationship, end node. So, a row in the result looks like:
    # [{n stuff}, {r stuff}, {m stuff}]
    #
    # The code below extracts the application data.
    #
    actor = get_stuff(record[0])
    rel = get_stuff(record[1])
    movie = get_stuff(record[2])
    print("\tActor = ", json.dumps(actor))
    print("\tRelationship = ", json.dumps(rel))
    print("\tMovie = ", json.dumps(movie))
    print("")
    

	Actor =  {"id": 71, "labels": ["Person"], "properties": {"born": 1956, "name": "Tom Hanks"}}
	Relationship =  {"id": 202, "labels": null, "properties": {"roles": ["Jim Lovell"]}}
	Movie =  {"id": 144, "labels": ["Movie"], "properties": {"tagline": "Houston, we have a problem.", "title": "Apollo 13", "released": 1995}}

	Actor =  {"id": 71, "labels": ["Person"], "properties": {"born": 1956, "name": "Tom Hanks"}}
	Relationship =  {"id": 84, "labels": null, "properties": {"roles": ["Joe Fox"]}}
	Movie =  {"id": 67, "labels": ["Movie"], "properties": {"tagline": "At odds in life... in love on-line.", "title": "You've Got Mail", "released": 1998}}

	Actor =  {"id": 71, "labels": ["Person"], "properties": {"born": 1956, "name": "Tom Hanks"}}
	Relationship =  {"id": 234, "labels": null, "properties": {"roles": ["Jimmy Dugan"]}}
	Movie =  {"id": 162, "labels": ["Movie"], "properties": {"tagline": "Once in a lifetime you get a chance to do something different.", "title": "A League of Their Own

- Running the query on the Neo4j console produces.

<img src="./more_complex_neo4j.png">

- You can choose the display result in ```Table``` option to see a non-graphical view of the result.

<img src="./table_example.png">

__Testing Some Cooler Stuff__

In [50]:
# Neo4j queries can return graph data: nodes, relationships, paths.
# You can also request the data in more traditional forms, e.g.
# a table of property values from the various nodes and relationships
# in the result set. The query below returns properties of the
# result objects, not the objects.
q =  """MATCH (n:Person)-[r:ACTED_IN]->(m:Movie) where n.name='Tom Hanks'
        return n.name,r.roles,m.title,m.released"""
result = gdb.query(q)

# Since this is a table, we can load into Pandas. Pandas is an incredibly
# common and powerful library for manipulating tabular data.
# https://pandas.pydata.org/
df = pandas.DataFrame(result, columns=["Name", "Roles", "Title", "Released"])    

# Display the data.
df
    

,Name,Roles,Title,Released
0,Tom Hanks,[Jim Lovell],Apollo 13,1995
1,Tom Hanks,[Joe Fox],You've Got Mail,1998
2,Tom Hanks,[Jimmy Dugan],A League of Their Own,1992
3,Tom Hanks,[Joe Banks],Joe Versus the Volcano,1990
4,Tom Hanks,[Mr. White],That Thing You Do,1996
5,Tom Hanks,[Dr. Robert Langdon],The Da Vinci Code,2006
6,Tom Hanks,"[Zachry, Dr. Henry Goose, Isaac Sachs, Dermot ...",Cloud Atlas,2012
7,Tom Hanks,[Chuck Noland],Cast Away,2000
8,Tom Hanks,[Paul Edgecomb],The Green Mile,1999
9,Tom Hanks,[Sam Baldwin],Sleepless in Seattle,1993


## Redis

- Connect to the PaaS version of Redis that you set up.


- You will have to use __USE THE URL, USER ID AND PASSWORD YOU CONFIGURED__ when <br> setting up your pass version of Redis.

In [85]:
# Import the Redis client package.
import redis
# Connect to Redis. You can see the "connections" are a recurring concept in databases.
# USE THE USERID, URL, ETC. YOU CONFIGURED FOR YOUR INSTANCE.
rd = redis.Redis(host="SG-shangzi-28810.servers.mongodirector.com", password="ieE8rImKQMhw5Num8bN5bOCMUWqAFwu9")

- ```result``` is a variable set above as the return from code calling Neo4j.

In [86]:
df.to_json()

'{"Follower":{"0":"George Washington","1":"John Adams","2":"John Adams","3":"Thomas Jefferson","4":"Thomas Jefferson","5":"James Monroe","6":"James Monroe","7":"James Madison","8":"James Madison"},"Followee":{"0":"Ted Williams","1":"Ted Williams","2":"George Washington","3":"Ted Williams","4":"John Adams","5":"Ted Williams","6":"James Madison","7":"Thomas Jefferson","8":"Ted Williams"}}'

- We will store the data in Redis. This requires a key and value.


- The value is complex and does not easily map to a core Redis data type. We are using Redis to "cache" results and do not need to access the sub-elements of the results. So, we simply store as a JSON string.


- We structure the key using a convention to allow us to find information. In this case, the convention is ```Person.name:Relationship.label.```


- We are serializing the data. https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/concepts/serialization/ is an explanation of serialization in C#. The same concepts apply here.

In [87]:
# Serialize the data into a string.
# 
data = df.to_json()
key = "Tom Hanks:ACTED_IN"
res = rd.set(key, data)
print("Set result = ", res)

Set result =  True


- ```True``` means that our set succeeded.


- Let's see if we can get it back. First, let's examine the keys in the database.

In [88]:
result = rd.keys("*")
print("Result =")
for r in result:
    print("\t", r)


Result =
	 b'Tom Hanks:ACTED_IN'


- The ```b``` stuff is the way Python displays the [string encoding.](https://stackoverflow.com/questions/6269765/what-does-the-b-character-do-in-front-of-a-string-literal) There are many encoding tables for strings, as we saw in MySQL.

- Let's use the key to retrieve our previous answer that we stored.

In [89]:
roles = rd.get(key)
print("Raw answer  as the giant string we stored = ", roles)

Raw answer  as the giant string we stored =  b'{"Follower":{"0":"George Washington","1":"John Adams","2":"John Adams","3":"Thomas Jefferson","4":"Thomas Jefferson","5":"James Monroe","6":"James Monroe","7":"James Madison","8":"James Madison"},"Followee":{"0":"Ted Williams","1":"Ted Williams","2":"George Washington","3":"Ted Williams","4":"John Adams","5":"Ted Williams","6":"James Madison","7":"Thomas Jefferson","8":"Ted Williams"}}'


- We know that the string is serialized JSON because we are the one who put it there.


- We can de-serialize and then load into a data frame.

In [90]:
roles = json.loads(roles)
roles = pandas.DataFrame(roles, columns=["Name", "Roles", "Title", "Released"])

In [91]:
roles

,Name,Roles,Title,Released


# HW4 $-$ Part 1: Neo4j

See documentation for questions: https://neo4j-rest-client.readthedocs.io/en/latest/info.html

## Creating Nodes

- The following people are both ```Person``` and ```Fan.``` Please create nodes for these people.
    - George Washington
    - Thomas Jefferson
    - James Madison
    - Abraham Lincoln
    - John F. Kennedy
    

In [51]:
name = [
    "George Washington",
    "John Adams",
    "Thomas Jefferson",
    "James Madison",
    "James Monroe",
    "Ted Williams"
]
q_s = "create (n:Person:Fan {{name:'{}'}})"

for n in name:
  c_q = q_s.format(n)
  gdb.query(c_q)
# Your create code goes here.

- After creation, run the following code. You should get the sample answer.

In [52]:
fan_q = "match (f:Fan) return f"
fans = []
fan_res = gdb.query(fan_q)

for f in fan_res:
    t = get_stuff(f[0])
    t = [t['id'], t['labels'], t['properties']['name']]
    fans.append(t)
    
fan_df = pandas.DataFrame(fans, columns=['Id', 'Labels', 'Name'])

In [53]:
fan_df

,Id,Labels,Name
0,186,"[Person, Fan]",George Washington
1,245,"[Person, Fan]",John Adams
2,246,"[Person, Fan]",Thomas Jefferson
3,265,"[Person, Fan]",James Monroe
4,266,"[Person, Fan]",Ted Williams
5,303,"[Person, Fan]",James Madison


## Creating Relationships

- Create ```Follows``` relationships:
    - John Adams ```Follows``` George Washing
    - Thomas Jefferson ```Follows``` John Adams
    - James Madison ```Follows``` Thomas Jefferson
    - James Monroe ```Follows``` James Madison
    - EVERYONE ```Follows``` Ted Williams

In [54]:
# John Adams Follows George Washing
q = """MATCH (a:Person:Fan),(b:Person:Fan)
WHERE a.name = 'John Adams' AND b.name = 'George Washington'
CREATE (a)-[r:Follows { name: a.name + 'Follows' + b.name }]->(b)
RETURN type(r), r.name"""

result = gdb.query(q)

In [55]:
# Thomas Jefferson Follows John Adams
q = """
MATCH (a:Person:Fan),(b:Person:Fan)
WHERE a.name = 'Thomas Jefferson' AND b.name = 'John Adams'
CREATE (a)-[r:Follows { name: a.name + 'Follows' + b.name }]->(b)
RETURN type(r), r.name
"""

result = gdb.query(q)

In [63]:
# James Madison Follows Thomas Jefferson
q = """MATCH (a:Person:Fan),(b:Person:Fan) 
WHERE a.name = 'James Madison' AND b.name = 'Thomas Jefferson'
CREATE (a)-[r:Follows { name: a.name + 'Follows' + b.name }]->(b)
RETURN type(r), r.name"""

result = gdb.query(q)

In [57]:
# James Monroe Follows James Madison
q = """MATCH (a:Person:Fan),(b:Person:Fan)
WHERE a.name = 'James Monroe' AND b.name = 'James Madison'
CREATE (a)-[r:Follows { name: a.name + 'Follows' + b.name }]->(b)
RETURN type(r), r.name"""

result = gdb.query(q)

In [58]:
# everyone follows Ted Williams
q = """MATCH (a:Person:Fan),(b:Person:Fan)
WHERE a.name = 'George Washington' AND b.name = 'Ted Williams'
CREATE (a)-[r:Follows { name: a.name + 'Follows' + b.name }]->(b)
RETURN type(r), r.name"""
result = gdb.query(q)

In [59]:
q = """MATCH (a:Person:Fan),(b:Person:Fan)
WHERE a.name = 'John Adams' AND b.name = 'Ted Williams'
CREATE (a)-[r:Follows { name: a.name + 'Follows' + b.name }]->(b)
RETURN type(r), r.name"""
result = gdb.query(q)

In [60]:
q = """MATCH (a:Person:Fan),(b:Person:Fan)
WHERE a.name = 'Thomas Jefferson' AND b.name = 'Ted Williams'
CREATE (a)-[r:Follows { name: a.name + 'Follows' + b.name }]->(b)
RETURN type(r), r.name"""
result = gdb.query(q)

In [61]:
q = """MATCH (a:Person:Fan),(b:Person:Fan)
WHERE a.name = 'James Madison' AND b.name = 'Ted Williams'
CREATE (a)-[r:Follows { name: a.name + 'Follows' + b.name }]->(b)
RETURN type(r), r.name"""
result = gdb.query(q)

In [62]:

q = """MATCH (a:Person:Fan),(b:Person:Fan)
WHERE a.name = 'James Monroe' AND b.name = 'Ted Williams'
CREATE (a)-[r:Follows { name: a.name + 'Follows' + b.name }]->(b)
RETURN type(r), r.name"""

result = gdb.query(q)

- After running your code, returning all of the ```Fans``` in the browser should look like:

<img src="./fans.png">



- You can also run the following test to check your answer:

In [64]:
q = "match (n:Fan)-[r:Follows]->(m) return n.name, m.name"
d = gdb.query(q)
df = pandas.DataFrame(d, columns=['Follower', 'Followee'])
df

,Follower,Followee
0,George Washington,Ted Williams
1,John Adams,Ted Williams
2,John Adams,George Washington
3,Thomas Jefferson,Ted Williams
4,Thomas Jefferson,John Adams
5,James Monroe,Ted Williams
6,James Monroe,James Madison
7,James Madison,Thomas Jefferson
8,James Madison,Ted Williams


- Create some ```Likes``` relationships:
    - Ted Williams ```Likes``` the ```Person``` Tom Hanks
    - James Monroe ```Likes``` the ```Person``` Robert Longo

In [65]:
q = """MATCH (a:Person),(b:Person)
WHERE a.name = 'Ted Williams' AND b.name = 'Tom Hanks'
CREATE (a)-[r:Likes { name: a.name + 'Likes' + b.name }]->(b)
RETURN type(r), r.name"""

result = gdb.query(q)

In [67]:
q= """MATCH (a:Person),(b:Person)
WHERE a.name = 'James Monroe' AND b.name = 'Robert Longo'
CREATE (a)-[r:Likes { name: a.name + 'Likes' + b.name }]->(b)
RETURN type(r), r.name """

result = gdb.query(q)

- After creating the relationships, examing ```Likes``` in the browser should return:

<img src="./likes.png">



- The following code will test your result:

In [68]:
q = "match (n)-[:Likes]->(m) return n.name, m.name"
res = gdb.query(q)
df_likes = pandas.DataFrame(res, columns=['Liker', 'Likee'])
df_likes

,Liker,Likee
0,Ted Williams,Tom Hanks
1,James Monroe,Robert Longo


## Spooky Query

- Find a shortest non-directed path from ```Person``` Robert Longo to ```Person``` Kevin Bacon. Running the query in the Neo4j browser will return:


<hr style="height:2px">
Write your query in here.

MATCH (cs:Person { name: 'Robert Longo' }),(ms:Person { name: 'Kevin Bacon' }), p = shortestPath((cs)-[*]-(ms))
WHERE length(p)> 1
RETURN p

<hr style="height:2px">

- Running your query in Neo4j Browser will produce:

<img src="./path.png">


- Submit your screen shot showing query in your submission.

<img src ="./screen.png">

# Redis

## Introduction

- One if, if not the most common, uses of Redis is for result caching.


- The following figure provides an overview of the concept.

| <img src="./redis_cache.png"> |
| :---: |
| [Redis Caching](https://www.sohamkamani.com/blog/2016/10/14/make-your-node-server-faster-with-redis-cache/) |


- You can find more details on the concept [here](https://www.sohamkamani.com/blog/2016/10/14/make-your-node-server-faster-with-redis-cache/)


- The basic concept is that the server code implements logic similar to the pseudo-code

```
def server(query):

    res = check_cache(query)
    if res is not None:
        return res
        
    db_res = db.query(query)
    cache_add(query, db_res)
    return db_res
```

- You are going to implement a simple version of the concept using Redis to cache in-front of Neo4j.


- We have seen the recurring pattern for queries of the form:

```
{
    "table_name": "t_name",
    "template": {
        "p1": "v1",
        "p2": "v1",
        ... ...
    }
}
```

- The concept represents simple queries in SQL, REST, ...


- The code below shows that you can do something similar with Neo4j.

In [92]:
#This is a Neo4j Query, not a redis query
def query_by_template(db, t_name, template):
    
    q = "match (n:" + t_name + ")  "
    
    terms = []
    for k,v in template.items():
        if type(v) == str:
            v = "'" + v + "'"
        terms.append("n."+ k + "=" + str(v))
        
    if len(terms) > 0:
        q += "where " + ' and '.join(terms)
        
    q += " return n "
    
    res = db.query(q)
    
    result = []
    for r in res:
        result.append(get_stuff(r[0]))
    return result

In [93]:
t_name = "Movie"
template = {"released": 2008}
res = query_by_template(gdb, t_name, template)
#print(res)

print(res)

[{'id': 121, 'labels': ['Movie'], 'properties': {'tagline': 'Speed has no limits', 'title': 'Speed Racer', 'released': 2008}}, {'id': 137, 'labels': ['Movie'], 'properties': {'tagline': '400 million people were waiting for the truth.', 'title': 'Frost/Nixon', 'released': 2008}}]


## Assignment

- We have seen that Redis stores data in many forms, one of which is key and string value.


- The following is a template for two functions that extend the example above to support caching in Redis

In [94]:
'''
see examples at the bottom for clarification on what constitutes a 
template and value.

Check cache should return a result
'''
def cache_it(template, value):
    k = json.dumps(template)
    v = json.dumps(value)
    rd.set(k,v)

def check_cache(template):
    k = json.dumps(template)
    res = rd.get(k)
    return res

- And this is a simple example of extending the query function above to exploit caching.


In [95]:
import copy

def query_by_template_cache(db, t_name, template):
    
    if template is not None:
        tt = t_name + ":" + json.dumps(template)
    else:
        tt = t_name
        
    r = check_cache(tt)
    
    if r is not None:
        print("Cache HIT!")
        return json.loads(r)
    else:
        print("Cache MISS!")
    
    q = "match (n:" + t_name + ")  "
    
    terms = []
    for k,v in template.items():
        if type(v) == str:
            v = "'" + v + "'"
        terms.append("n."+ k + "=" + str(v))
        
    if len(terms) > 0:
        q += "where " + ' and '.join(terms)
        
    q += " return n "
    
    res = db.query(q)
    
    result = []
    for r in res:
        result.append(get_stuff(r[0]))
        
    cache_it(tt, json.dumps(result))
    
    return result

- Implement the functions ```check_cache``` and ```cache_hit.```


- You can test your functions below.

In [96]:
t_name = "Movie"
template = {"released": 2008, "title": "Frost/Nixon"}
res = query_by_template_cache(gdb, t_name, template)
print(res)

Cache MISS!
[{'id': 137, 'labels': ['Movie'], 'properties': {'tagline': '400 million people were waiting for the truth.', 'title': 'Frost/Nixon', 'released': 2008}}]


In [97]:
t_name = "Movie"
template = {"released": 2008, "title": "Frost/Nixon"}
res = query_by_template_cache(gdb, t_name, template)
print(res)

Cache HIT!
[{"id": 137, "labels": ["Movie"], "properties": {"tagline": "400 million people were waiting for the truth.", "title": "Frost/Nixon", "released": 2008}}]
